In [1]:
import warnings
import numpy as np

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from importlib import reload

import wandb

from context import omphalos

# Import Omphalos modules.
from omphalos import generate_inputs as gi
from omphalos import file_methods as fm
from omphalos import my_metrics as mm
from omphalos import omphalos_plotter as op
from omphalos import attributes as attr
from omphalos import labels as lbls
from omphalos import spatial_constructor

from matplotlib import pyplot as plt

import xgboost as xgb

%matplotlib osx
%precision 4

'%.4f'

In [2]:
bst = xgb.Booster()
bst.load_model(fname='1086_model.json')

In [3]:
%%time

species_list = ['SO4--','Ca++', 'Formaldehyde', 'CO2(aq)']
axis_labels = ['[SO$_4^{2-}$] (mM)','[Ca$^{2+}$] (mM)', '[Formaldehyde] (mM)', '[CO$_{2(aq)}$] (mM)']

def plot_2d(ax, plot_vars):
    dims = 4
    num = 20
    upper = 0.05
    lower = 0
    x1 = np.linspace(lower, upper, num) # NH4+
    x2 = np.linspace(lower, upper, num) # SO4--
    x1_mesh,x2_mesh = np.meshgrid(x1, x2)
    X = np.dstack([x1_mesh, x2_mesh]).reshape(-1, 2)

    defaults = np.zeros((num**2, 4))
    defaults[:,0] = 26.79609e-03 # SO4--
    defaults[:,1] = 1.01838314236E-02 # Ca++
    defaults[:,2] = 0.55e-01 # Formaldehyde
    defaults[:,3] = 2.00e-03 # CO2(aq)
    
    defaults[:, plot_vars[0]] = X[:, 0]
    defaults[:, plot_vars[1]] = X[:, 1]

    vals = bst.predict(xgb.DMatrix(defaults))

    ax.plot_trisurf(X[:,0] * 1000, X[:,1] * 1000, vals*1e-4, cmap=plt.cm.magma, linewidth=0, antialiased=False)
    
    ax.set_xlabel(axis_labels[plot_vars[0]], fontsize=18, labelpad=7)
    ax.set_ylabel(axis_labels[plot_vars[1]], fontsize=18, labelpad=7)
    ax.set_zlabel('Net calcite precpitation (vol. frac.)', fontsize=18, labelpad=29)
    ax.tick_params('x', labelsize=15, pad=0)
    ax.tick_params('y', labelsize=15, pad=0)
    ax.tick_params('z', labelsize=15, pad=15)
    #ax.ticklabel_format(axis='z', style='sci', scilimits=(0,0))
    ax.view_init(elev=15, azim=-128)
    return

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 32.9 µs


In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import string

plt.style.use('seaborn-paper')


axis_labels = ['[SO$_4^{2-}$] (mM)','[Ca$^{2+}$] (mM)', '[Formaldehyde] (mM)', '[CO$_{2(aq)}$] (mM)']

fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(231, projection='3d')
ax2 = fig.add_subplot(232, projection='3d')
ax3 = fig.add_subplot(233, projection='3d')
ax4 = fig.add_subplot(234, projection='3d')
ax5 = fig.add_subplot(235, projection='3d')
ax6 = fig.add_subplot(236, projection='3d')

ax_list = [ax1, ax2, ax3, ax4, ax5, ax6]
#ax_list = [ax1, ax2, ax3]

plot_list = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]



for ax, i, n in zip(ax_list, plot_list, np.arange(len(plot_list))):
        plot_2d(ax, i)
        ax.text2D(-0.1, 1.1, '({})'.format(string.ascii_uppercase[n]), transform=ax.transAxes, size=20)
    

#plt.subplots_adjust(wspace=0.25, hspace=0.25)


In [7]:
fig.savefig('/Users/angus/Dropbox/dissertation/1086_2d.png', dpi=300)

In [17]:
%%time

#Finding best parameter set!
num = 15
upper = 0.05
lower = 0
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x3 = np.linspace(lower, upper, num)
x4 = np.linspace(lower, upper, num)

XX = np.stack(np.meshgrid(x1, x2, x3, x4), -1).reshape(-1, 4)

vals = bst.predict(xgb.DMatrix(XX))

max_index = np.argmax(vals)

print(max_index)
print(XX[max_index])
print(vals[max_index]*1e-4)

45674
[0.025  0.0464 0.05   0.05  ]
0.0015149385452270508
CPU times: user 16min 54s, sys: 36.2 s, total: 17min 30s
Wall time: 2min 48s
